## Introduction

Ce notebook jupyter réalise le test du réseau en prenant en argument le nom d'un répertoire d'images à classer et générera un fichier csv du même type que ceux présents dans " Données ";
Ce notebook a été réalisé sur google colab par : Yahya Abourriche.

##Préparation de données
On mount notre drive afin de pouvoir accéder à notre model.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Afin de pouvoir accéder aux fichiers, on décompresse notre fichier .zip et on met son contenu dans le chemin standard.

In [7]:
!unzip drive/MyDrive//Data_Train_Valid.zip > /dev/null

On commence par importer notre model créé sous forme h5

In [18]:
from tensorflow import keras
model = keras.models.load_model("drive/MyDrive/mon_model_h5.h5")

On ouvre toutes les images présentes dans le fichier ouvert et on les mets dans une liste.

On met les noms des images dans une autre liste.

In [19]:
import glob
from PIL import Image
import numpy as np
image_list = []
nom_list = []
nomd = 'images/hand/'
for filename in glob.glob(nomd + '*.png'): 
    im=Image.open(filename)
    image_list.append(im)
    nom_list.append(filename)

On transforme les images en listes et on les mets dans une liste


In [20]:
from tensorflow.keras.preprocessing.image import img_to_array
X=[]
for i in range(0,len(image_list)):
  image = img_to_array(image_list[i])
  X.append(image)
len(X)

20000

On transforme la liste contenant les images en une numpy *array*

In [21]:
X = np.array(X, dtype="float") / 255.0

On peut alors utiliser notre model afin de prédire la classification des images qu'on a obtenu :

In [25]:
pred = model.predict(X)
pred[:10]

array([[ -7.098916  ,  -1.6713388 ,  -6.2696853 ,  -5.3833594 ,
         18.057007  ],
       [ -9.770866  ,  -8.216786  , -13.93754   ,   0.36183417,
         18.675346  ],
       [ -6.0906553 ,  -3.3640347 , -12.870415  ,   2.7149653 ,
         11.885069  ],
       [ -5.8182774 ,  -3.0776615 ,  -2.129076  ,  -2.1082487 ,
         10.927973  ],
       [  0.97554535,  -1.8353299 ,   1.926396  ,  -1.1270304 ,
          0.10560912],
       [  0.7687864 ,  -6.1951947 ,  -1.4622022 ,  -4.043907  ,
          7.505346  ],
       [ -7.242484  ,  -6.109723  ,  -6.060951  ,   2.376879  ,
          9.414477  ],
       [ -7.8135605 ,  -2.8211663 ,  -5.350155  ,  -6.5962067 ,
         14.834732  ],
       [ -5.591817  ,  -5.3213997 ,  -8.351862  ,  -5.3957963 ,
         16.732836  ],
       [ -8.019685  ,  -1.9759357 , -10.68922   ,  -6.0423875 ,
         21.350874  ]], dtype=float32)

On obtient des résultats sous forme de probabilité.

Afin d'obtenir des résultats cohérents, il faut utiliser la fonction argmax de numpy pour retrouver un seul résultat :

In [26]:
predcl = np.argmax(pred, axis = 1)
predcl[:10]

array([4, 4, 4, 4, 2, 4, 4, 4, 4, 4])

On peut alors créer un fichier sous format csv de même manière que les fichiers train.csv et valid.csv .

Pour chaque image présentes dans le dossier, on ajoute le nom de l'image, le chemin relatif et la *class_label*

In [28]:
# contenu du fichier csv 
row=[['image_name','relative_path','class_label']]
for i in range(0,len(X)):
  lis = nom_list[i].split('/')
  namec = lis[len(lis)-1]
  pathc = lis[len(lis)-2] + '/' + namec
  row.append([namec,pathc, predcl[i]])

# ecriture ss format csv
np.savetxt("sorti.csv", 
           row,
           delimiter =", ", 
           fmt ='% s')